In [ ]:
import pandas as pd
import warnings
import locale
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import unicodedata
import numpy as np
import geopandas as gpd
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:.3f}'.format
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
%%sql
SELECT 
    data, 
    numeroNotaFiscal, 
    destino, 
    procedencia, 
    cad_cidades.estado as estado,
    produto, 
    unidade, 
    unidade_peso, 
    quantidade, 
    volume,
    cad_usuarios.tipo,
    nome_usuario_sistema
FROM 
    divtec_notasfiscais
LEFT JOIN
    cad_usuarios
        ON cad_usuarios.matricula_nome = divtec_notasfiscais.destino
LEFT JOIN
    cad_cidades ON divtec_notasfiscais.procedencia = cad_cidades.cidade
WHERE data >= '2024-09-16' and data < '2024-09-20'

### Cotação

In [ ]:
%%sql
CALL cotacaoEmQuilogramasPorPeriodo('2024-09-16','2024-09-19')

In [ ]:
df_cotacao.rename(columns={'Preço por Kg':'preco'}, inplace=True)

In [ ]:
dados = df.merge(right=df_cotacao, on=['produto','data'], how='left')
dados = dados[['data','destino','procedencia','estado','produto','volume','tipo','preco']]
dados

Listando as linhas com dados faltantes referente a preços

In [ ]:
dados[['data','produto','preco']].loc[dados['preco'].isnull()]

In [ ]:
dados['valor'] = dados['volume'] * dados['preco']
dados['data'] = dados['data'].astype('datetime64[ns]')

In [ ]:
dados

## Análise
<hr>

### Qual foi o volume e valor comercializado por Produtores e Atacadistas na semana?

<strong><i>ATACADISTAS</i></strong>

In [ ]:
dados_atacadistas = dados.copy(deep=True)
dados_atacadistas = dados_atacadistas.loc[dados_atacadistas['tipo'] == 'ATACADISTA']

In [ ]:
dados_atacadistas = dados_atacadistas.groupby(['data'])[['volume','valor']].sum().reset_index()

Gráfico de Volume

In [ ]:
def format_decimal(x, pos):
	return f'{x:.2f} kg'.replace('.', ',')

fig, ax = plt.subplots()
sns.set_theme(style="white")

# Gerando o gráfico de colunas
ax = sns.barplot(x="data", y="volume", data=dados_atacadistas, palette="Blues", ci=None)

sns.despine(bottom=True, left=True)

title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'gray'}
ax.set_title("Volume Comercializado por Atacadistas no período\n16/09 a 19/09", pad=20, fontdict=title_font, loc='left')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels("")

# Set x-axis tick labels to day of month
labels = [16,17,18,19]
ax.set_xticklabels(labels)

for p in ax.patches:
	volume = p.get_height()
	ax.text(p.get_x() + p.get_width()/2, volume, f'{volume:,.2f} kg'.replace(',', 'v').replace('.', ',').replace('v', '.'), ha='center', va='bottom', color='grey', fontsize=12, fontweight='bold')


plt.tight_layout()
plt.show()

Gráfico de Valor

In [ ]:
def format_decimal(x, pos):
	return f'R$ {x:.2f} kg'.replace('.', ',')

fig, ax = plt.subplots()
sns.set_theme(style="white")

# Gerando o gráfico de colunas
ax = sns.barplot(x="data", y="valor", data=dados_atacadistas, palette='Greens', ci=None)

sns.despine(bottom=True, left=True)

title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'gray'}
ax.set_title("Valor Comercializado por Atacadistas no período\n16/09 a 19/09", pad=20, fontdict=title_font, loc='left')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels("")

# Set x-axis tick labels to day of month
labels = [16,17,18,19]
ax.set_xticklabels(labels)

for p in ax.patches:
	valor = p.get_height()
	ax.text(p.get_x() + p.get_width()/2, valor, f'R$ {valor:,.2f}'.replace(',', 'v').replace('.', ',').replace('v', '.'), ha='center', va='bottom', color='grey', fontsize=12, fontweight='bold')


plt.tight_layout()
plt.show()

<strong><i>PRODUTORES</i></strong>

In [ ]:
dados_produtores = dados.copy(deep=True)
dados_produtores = dados_produtores.loc[dados_produtores['tipo'] == 'PRODUTOR']

In [ ]:
dados_produtores = dados_produtores.groupby(['data'])[['volume','valor']].sum().reset_index()

In [ ]:
def format_decimal(x, pos):
	return f'{x:.2f} kg'.replace('.', ',')

fig, ax = plt.subplots()
sns.set_theme(style="white")

# Gerando o gráfico de colunas
ax = sns.barplot(x="data", y="volume", data=dados_produtores, palette="Blues", ci=None)

sns.despine(bottom=True, left=True)

title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'gray'}
ax.set_title("Volume Comercializado por Produtores no período\n16/09 a 19/09", pad=20, fontdict=title_font, loc='left')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels("")

# Set x-axis tick labels to day of month
labels = [16,17,18,19]
ax.set_xticklabels(labels)

for p in ax.patches:
	volume = p.get_height()
	ax.text(p.get_x() + p.get_width()/2, volume, f'{volume:,.2f} kg'.replace(',', '.'), ha='center', va='bottom', color='grey', fontsize=12, fontweight='bold')


plt.tight_layout()
plt.show()

In [ ]:
def format_decimal(x, pos):
	return f'R$ {x:.2f} kg'.replace('.', ',')

fig, ax = plt.subplots()
sns.set_theme(style="white")

# Gerando o gráfico de colunas
ax = sns.barplot(x="data", y="valor", data=dados_produtores, palette='Greens', ci=None)

sns.despine(bottom=True, left=True)

title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'gray'}
ax.set_title("Valor Comercializado por Produtores no período\n16/09 a 19/09", pad=20, fontdict=title_font, loc='left')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels("")

# Set x-axis tick labels to day of month
labels = [16,17,18,19]
ax.set_xticklabels(labels)

for p in ax.patches:
	valor = p.get_height()
	ax.text(p.get_x() + p.get_width()/2, valor, f'R$ {valor:,.2f}'.replace(',', 'v').replace('.', ',').replace('v', '.'), ha='center', va='bottom', color='grey', fontsize=12, fontweight='bold')


plt.tight_layout()
plt.show()

### Qual os 5 produtos com maior volume comercializado e valor? (Atacadistas, Produtores e no geral)

In [ ]:
dados_diario = dados.copy(deep=True)
dados_diario = dados_diario.groupby(['data'])[['volume', 'valor']].sum().reset_index()

In [ ]:
def format_decimal(x, pos):
	return f'{x:.2f} kg'.replace('.', ',')

fig, ax = plt.subplots()
sns.set_theme(style="white")

# Gerando o gráfico de colunas
ax = sns.barplot(x="data", y="volume", data=dados_diario, palette="Reds", ci=None)

sns.despine(bottom=True, left=True)

title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'gray'}
ax.set_title("Volume Comercializado no período\n16/09 a 19/09", pad=20, fontdict=title_font, loc='left')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels("")

# Set x-axis tick labels to day of month
labels = ['Segunda','Terça','Quarta','Quinta']
ax.set_xticklabels(labels)

for p in ax.patches:
	volume = p.get_height()
	ax.text(p.get_x() + p.get_width()/2, volume, f'{volume:,.2f} kg'.replace(',', '.'), ha='center', va='bottom', color='grey', fontsize=12, fontweight='bold')


plt.tight_layout()
plt.show()

In [ ]:
def format_decimal(x, pos):
	return f'R$ {x:.2f} kg'.replace('.', ',')

fig, ax = plt.subplots()
sns.set_theme(style="white")

# Gerando o gráfico de colunas
ax = sns.barplot(x="data", y="valor", data=dados_diario, palette='Greys', ci=None)

sns.despine(bottom=True, left=True)

title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'gray'}
ax.set_title("Valor Comercializado por Produtores no período\n16/09 a 19/09", pad=20, fontdict=title_font, loc='left')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels("")

# Set x-axis tick labels to day of month
labels = ['Segunda','Terça','Quarta','Quinta']
ax.set_xticklabels(labels)

for p in ax.patches:
	valor = p.get_height()
	ax.text(p.get_x() + p.get_width()/2, valor, f'R$ {valor:,.2f}'.replace(',', 'v').replace('.', ',').replace('v', '.'), ha='center', va='bottom', color='grey', fontsize=12, fontweight='bold')


plt.tight_layout()
plt.show()

### Qual o volume e valor comercializado por produto

In [ ]:
dados_produtos = dados.copy(deep=True)
dados_produtos = dados_produtos.groupby(['produto'])[['volume','valor']]


In [ ]:
def format_decimal(x, pos):
	return f'{x:.2f} kg'.replace('.', ',')

fig, ax = plt.subplots()
sns.set_theme(style="white")

# Gerando o gráfico de colunas
ax = sns.barplot(x="data", y="volume", data=dados_diario, palette="Reds", ci=None)

sns.despine(bottom=True, left=True)

title_font = {'fontsize': 16, 'fontweight': 'bold', 'color': 'gray'}
ax.set_title("Volume Comercializado no período\n16/09 a 19/09", pad=20, fontdict=title_font, loc='left')
ax.set_xlabel("")
ax.set_ylabel("")
ax.set_yticklabels("")

# Set x-axis tick labels to day of month
labels = ['Segunda','Terça','Quarta','Quinta']
ax.set_xticklabels(labels)

for p in ax.patches:
	volume = p.get_height()
	ax.text(p.get_x() + p.get_width()/2, volume, f'{volume:,.2f} kg'.replace(',', '.'), ha='center', va='bottom', color='grey', fontsize=12, fontweight='bold')


plt.tight_layout()
plt.show()